In [2]:
from langchain_core.output_parsers import StrOutputParser
from llm_provider import get_llm
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain.prompts import PromptTemplate

In [3]:
def stream_res(response):
    for token in response:
        print(token, end="", flush=True)

In [4]:
# llm = get_llm(provider="ollama",model="llama3.2")

## openai
llm = get_llm()

In [5]:
url ="https://n.news.naver.com/article/437/0000378416"
# url ="https://n.news.naver.com/mnews/article/421/0008117615"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)


docs = loader.load()
print(f"문서의 수: {len(docs)}")
print(docs)

문서의 수: 1
[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출

In [5]:

prompt_template = """Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. DO NOT include any unnecessary information.
5. Write a summary of the insights in KOREAN.

CONTEXT:
{context}

SUMMARY:"
"""



In [6]:
prompt = PromptTemplate.from_template(prompt_template)
chain = prompt |llm| StrOutputParser()

In [7]:
query = {"context": docs}

response = chain.stream(query)
stream_res(response)

Here is a summary of the article in bullet points with emojis:

• 💸 정부가 매달 주는 부모 급여 100만원과 첫만남이용권, 아동수당을 포함하여 아이 돌까지 1년 동안 1520만원을 받을 수 있습니다.
 
• 🤝 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살할 때까지 7400만원 주겠다고 했습니다.

• 💰 이와 같이 empresas들도 뛰어들고 있습니다. 한 회사는 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.

• 🏠 국민주택을 제공하겠다는 뜻도 밝혔습니다. 

• 📝 여력이 부족한 중소지원이 필요하다는 목소리도 나옵니다.

Here is a summary of the article in Korean:

👶 정부가 매달 주는 부모 급여 100만원과 첫만남이용권, 아동수당을 포함하여 아이 돌까지 1년 동안 1520만원을 받을 수 있습니다. 

🤝 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살할 때까지 7400만원 주겠다고 했습니다.

💰 이와 같이 empresas들도 뛰어들고 있습니다. 한 회사는 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.

🏠 국민주택을 제공하겠다는 뜻도 밝혔습니다. 

📝 여력이 부족한 중소지원이 필요하다는 목소리도 나옵니다.

In [8]:
prompt_template2 =  """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (100 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""


In [66]:
prompt2 = PromptTemplate.from_template(prompt_template2)
chain2 = prompt2 |llm| StrOutputParser()

In [67]:
query = {"ARTICLE": docs}
# print(query)

response = chain2.stream(query)
stream_res(response)

```json
[
    {
        "Missing_Entities": "부영그룹; 이상화 기자; 3년",
        "Denser_Summary": "출산 직원에게 1억원을 지급하는 파격적인 저출생 정책이 대두되고 있다. 정부의 저출생 대책으로 0세 아이 부모에게 월 100만원의 급여가 주어지며, 초기 아이 돌봄 비용으로 1520만원이 지원된다. 지자체에서도 인천시와 광주시가 각각 1억원과 7400만원을 지원하겠다고 밝혔고, 이는 과거 정치 공약과는 다른 진정성이 담겼다. 기업도 저출생 대책에 동참하고 있으며, 부영그룹이 2021년 이후 태어난 직원 자녀에게 1억원을 지급하는 정책을 시행키로 했다. 이상화 기자에 따르면, 이는 총 70억원 규모이다. 3년 이내에 3명이 태어날 경우 주택 제공도 예정되어 있다. 이 같은 출산 장려책은 법정 육아 휴직을 연장하고, 남성 직원의 육아휴직 의무화로 이어지고 있다. "
    },
    {
        "Missing_Entities": "장려금; 의료비; 네쌍둥이",
        "Denser_Summary": "부영그룹의 저출산 정책이 출산 직원에게 1억원을 지급하여 저출생 문제 해결을 추진하고 있다. 정부는 0세 아이 부모에게 매월 100만원을 지급하고, 지자체들은 인천시와 광주시에서 각각 1억원과 7400만원 지원을 약속하고 있다. 이러한 정책은 과거 정치적 공약과는 다른 실제적 변화로 반기고 있다. 특히 부영그룹은 2021년 이후 출생 고객 자녀에게 총 70억원을 지원하며, 3년 내에 3명 아이를 낳는 직원에게는 주택 제공할 예정이다. 직원들은 장려금을 통해 경제적 부담을 덜게 되고, 한 회사에서는 네쌍둥이를 낳은 직원에게 의료비를 지원했다는 사실이 주목받고 있다. "
    },
    {
        "Missing_Entities": "아이 돌; 아동수당; 광주시",
        "Denser_Summary": "부영그룹의 저출산 정책이 출산 직원에게 1억원 지급하며 주목받고 있다. 